### Imports

In [34]:
import cv2
import os

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [24]:
gesture_labels = {
    1: "Doing other things", 
    2: "Drumming Fingers", 
    3: "No gesture", 
    4: "Pulling Hand In", 
    5: "Pulling Two Fingers In", 
    6: "Pushing Hand Away", 
    7: "Pushing Two Fingers Away", 
    8: "Rolling Hand Backward", 
    9: "Rolling Hand Forward", 
    10: "Shaking Hand", 
    11: "Sliding Two Fingers Down",
    12: "Sliding Two Fingers Left", 
    13: "Sliding Two Fingers Right", 
    14: "Sliding Two Fingers Up", 
    15: "Stop Sign",
    16: "Swiping Down", 
    17: "Swiping Left", 
    18: "Swiping Right", 
    19: "Swiping Up", 
    20: "Thumb Down",
    21: "Thumb Up", 
    22: "Turning Hand Clockwise", 
    23: "Turning Hand Counterclockwise", 
    24: "Zooming In With Full Hand",
    25: "Zooming In With Two Fingers", 
    26: "Zooming Out With Full Hand", 
    27: "Zooming Out With Two Fingers"
}

In [25]:
def process_dataset(dataset_path):
    data = []
    labels = []

    for folder_number in range(1, 100000):  # Iterate over folder numbers from 1 to 99999
        gesture_name = gesture_labels.get(folder_number, None)  # Get gesture name from mapping
        if gesture_name is None:
            continue  # Skip folders without a gesture label

        gesture_path = os.path.join(dataset_path, str(folder_number))
        
        if os.path.isdir(gesture_path):  # Ensure it's a directory
            for img_file in os.listdir(gesture_path):
                img_path = os.path.join(gesture_path, img_file)
                img = cv2.imread(img_path)

                # Preprocess the image (resize, normalize)
                img_resized = cv2.resize(img, (64, 64))  # Adjust size as needed
                img_normalized = img_resized / 255.0
                data.append(img_normalized)
                labels.append(folder_number)

    return np.array(data), np.array(labels)

dataset_path = "../dataset/20bn-jester-v1"
X, y = process_dataset(dataset_path)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [35]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64,64,3)),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(128, (3,3), activation='relu'),  # Added another convolutional layer
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(256, activation='relu'),  # Increased hidden layer size
    Dropout(0.5),
    Dense(27, activation='softmax')
])

model.compile(
    optimizer=Adam(learning_rate=0.0001),  # Specify learning rate
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

In [ ]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

history = model.fit(
    X_train, 
    y_train_encoded, 
    epochs=10, 
    validation_data=(X_test, y_test_encoded), 
    batch_size=32
)

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 214ms/step - accuracy: 0.0413 - loss: 3.2959 - val_accuracy: 0.0309 - val_loss: 3.2960
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 165ms/step - accuracy: 0.0424 - loss: 3.2958 - val_accuracy: 0.0412 - val_loss: 3.2963
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 172ms/step - accuracy: 0.0386 - loss: 3.2958 - val_accuracy: 0.0412 - val_loss: 3.2964
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 170ms/step - accuracy: 0.0330 - loss: 3.2956 - val_accuracy: 0.0412 - val_loss: 3.2966
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 164ms/step - accuracy: 0.0394 - loss: 3.2955 - val_accuracy: 0.0412 - val_loss: 3.2970
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 178ms/step - accuracy: 0.0345 - loss: 3.2954 - val_accuracy: 0.0412 - val_loss: 3.2979
Epoch 7/10
 1/25 ━━━━━━━━━━━━━━━━━━━━ 4s 189ms/step - accuracy: 0.0312 - loss: 3.2946

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.2f}")

In [ ]:
model.save('gesture_recognition_model.h5')